<a href="https://colab.research.google.com/github/aymuos/masters-practise-repo/blob/main/TERM3/AI_at_Scale/Assignment1/question1/q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

q1

In [6]:
! pip install -q pyspark

In [13]:
# imports
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import col, sum

base_path = f'./'

#base_path = f"/opt/spark/data/{student_id}/"


In [9]:

spark = SparkSession.builder \
    .appName("assignment1_CH24M571") \
    .master("local[*]") \
    .config("spark.executor.memory", "512m") \
    .config("spark.executor.core", "4") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.instances", "2")\
    .config("spark.driver.core", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("INFO")


# # spark = SparkSession.builder \: This starts the process of building a SparkSession.
# .appName("Assignment1") \: This sets the name of the Spark application to "Assignment1".
# .master("local[*]") \: This configures Spark to run locally on your machine using all available cores.
# .config("spark.executor.memory", "512m") \: This sets the amount of memory allocated to each executor (the processes that perform the actual computations) to 512 megabytes.
# .config("spark.executor.core", "4") \: This sets the number of cores used by each executor to 4.
# .config("spark.driver.memory", "2g") \: This sets the amount of memory allocated to the Spark driver (the process that coordinates the execution) to 2 gigabytes.
# .config("spark.executor.instances", "2")\: This sets the number of executor instances to 2.
# .config("spark.driver.core", "2") \: This sets the number of cores used by the driver to 2.
# .getOrCreate(): This retrieves an existing SparkSession if one is already running, or creates a new one if not.
# spark.sparkContext.setLogLevel("INFO"): This sets the logging level for the Spark context to "INFO", which means informational messages will be displayed.

In [38]:
# setting the schemas for the 3 csv s

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType, StructType

# Customers schema
customers_schema = StructType([
    StructField("customer_id", StringType(), nullable=False),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("email", StringType(), True),
    StructField("preferences", StringType(), True),  # Will parse to ArrayType later
    StructField("address", StringType(), True)       # Will parse to StructType later
])

# Orders schema
orders_schema = StructType([
    StructField("order_id", StringType(), nullable=False),
    StructField("customer_id", StringType(), True),
    StructField("order_date", StringType(), True),
    StructField("items", StringType(), True),
    StructField("shipping_address", StringType(), True),
    StructField("total_amount", FloatType(), True)
])

# Products schema
products_schema = StructType([
    StructField("product_id", StringType(), nullable=False),
    StructField("name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", FloatType(), True),
    StructField("tags", StringType(), True)
])

In [39]:
# defining file paths

customers_csv= base_path +'customers.csv'
orders_csv = base_path +'orders.csv'
products_csv = base_path +'products.csv'

In [40]:

# read csv file
customers_df = spark.read.csv(customers_csv, header=True, schema=customers_schema,escape="\"",quote="\"")
# show the dataframe
customers_df.show()

+-----------+-----------+---+--------------------+--------------------+--------------------+
|customer_id|       name|age|               email|         preferences|             address|
+-----------+-----------+---+--------------------+--------------------+--------------------+
| CUST000001| Customer_1| 61|customer1@example...|["sports", "trave...|{"street": "2796 ...|
| CUST000002| Customer_2| 55|customer2@example...|                NULL|{"street": "5409 ...|
| CUST000003| Customer_3| 79|customer3@example...|["movies", "books...|{"street": "6081 ...|
| CUST000004| Customer_4| 73|customer4@example...|["books", "fitnes...|{"street": "3303 ...|
| CUST000005| Customer_5| 80|customer5@example...|["music", "books"...|{"street": "4864 ...|
| CUST000006| Customer_6| 44|customer6@example...|["travel", "fitne...|{"street": "3592 ...|
| CUST000007| Customer_7| 57|customer7@example...|["music", "movies...|{"street": "2932 ...|
| CUST000008| Customer_8| 20|                NULL|["books", "fitnes...

In [41]:
# read csv file
orders_df = spark.read.csv(orders_csv,header=True, schema=orders_schema,escape="\"",quote="\"")
# show the dataframe
orders_df.show()

+----------+-----------+----------+--------------------+--------------------+------------+
|  order_id|customer_id|order_date|               items|    shipping_address|total_amount|
+----------+-----------+----------+--------------------+--------------------+------------+
|ORD0000001| CUST000046|2023-08-16|[{"product_id": "...|{"street": "703 M...|     1112.73|
|ORD0000002| CUST000027|2023-11-20|[{"product_id": "...|{"street": "6503 ...|      456.04|
|ORD0000003| CUST000093|2023-11-27|[{"product_id": "...|{"street": "9664 ...|     1420.89|
|ORD0000004| CUST000017|2023-04-08|[{"product_id": "...|{"street": "1329 ...|     1450.66|
|ORD0000005| CUST000092|2023-09-24|[{"product_id": "...|{"street": "3075 ...|     2016.47|
|ORD0000006| CUST000029|2023-09-04|[{"product_id": "...|{"street": "176 M...|      800.88|
|ORD0000007| CUST000093|2023-10-22|[{"product_id": "...|{"street": "9664 ...|        NULL|
|ORD0000008| CUST000094|2023-10-20|[{"product_id": "...|{"street": "4391 ...|      121.35|

In [42]:
# read csv file
products_df = spark.read.csv(products_csv, header=True, schema=products_schema,escape="\"",quote="\"")
# show the dataframe
products_df.show()

+----------+----------+-----------+------+--------------------+
|product_id|      name|   category| price|                tags|
+----------+----------+-----------+------+--------------------+
| PROD00001| Product_1|electronics|  5.32|["new", "sale", "...|
| PROD00002| Product_2|     sports| 59.11|["limited", "eco-...|
| PROD00003| Product_3|     sports|370.91|             ["new"]|
| PROD00004| Product_4|     sports| 69.84|            ["sale"]|
| PROD00005| Product_5|     sports| 135.9|["sale", "new", "...|
| PROD00006| Product_6|electronics|428.33|["new", "limited"...|
| PROD00007| Product_7|electronics|400.44|         ["popular"]|
| PROD00008| Product_8|     sports| 40.45|["limited", "popu...|
| PROD00009| Product_9|     sports|187.57|["limited", "new"...|
| PROD00010|Product_10|  furniture| 176.2|  ["popular", "new"]|
+----------+----------+-----------+------+--------------------+



- Print the size of each dataframe and schema

In [44]:
print(f"Rows: {customers_df.count()}")
print(f"Columns: {len(customers_df.columns)}")
customers_df.printSchema()
print("-------------------------------------")
print(f"Rows: {orders_df.count()}")
print(f"Columns: {len(orders_df.columns)}")
orders_df.printSchema()
print("-------------------------------------")
print(f"Rows: {products_df.count()}")
print(f"Columns: {len(products_df.columns)}")
products_df.printSchema()
print("-------------------------------------")

Rows: 100
Columns: 6
root
 |-- customer_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- email: string (nullable = true)
 |-- preferences: string (nullable = true)
 |-- address: string (nullable = true)

-------------------------------------
Rows: 500
Columns: 6
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- items: string (nullable = true)
 |-- shipping_address: string (nullable = true)
 |-- total_amount: float (nullable = true)

-------------------------------------
Rows: 10
Columns: 5
root
 |-- product_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: float (nullable = true)
 |-- tags: string (nullable = true)

-------------------------------------


-
Identify the corrupt values in each file and report the number of corrupt rows and
total corrupt values in each file.

In [ ]:
# null check

from pyspark

## Identify null values

### Subtask:
Check for null values in columns that are expected to be non-nullable (like the ID columns).


**Reasoning**:
Import necessary functions and check for null values in the ID columns of each DataFrame as instructed.



In [46]:
# Check for null values in customers_df

# CHECKING JUST THE IDS - IDEALLY THERE SHOULDNT BE NONE

customers_null_counts = customers_df.select(
    sum(col("customer_id")
    .isNull()         # check if each value in column is null , if so cast the value as 0 or 1
    .cast("integer"))
    .alias("customer_id_nulls")).collect()[0]  # aggregate and put into a new column

    #
print(f"Customers DataFrame - Nulls in customer_id: {customers_null_counts['customer_id_nulls']}")

# Check for null values in orders_df

orders_null_counts = orders_df.select(
    sum(col("order_id").isNull().cast("integer")).alias("order_id_nulls"),
    sum(col("customer_id").isNull().cast("integer")).alias("customer_id_nulls")
).collect()[0]

print(f"Orders DataFrame - Nulls in order_id: {orders_null_counts['order_id_nulls']}")
print(f"Orders DataFrame - Nulls in customer_id: {orders_null_counts['customer_id_nulls']}")


# Check for null values in products_df
products_null_counts = products_df.select(sum(col("product_id").isNull().cast("integer")).alias("product_id_nulls")).collect()[0]
print(f"Products DataFrame - Nulls in product_id: {products_null_counts['product_id_nulls']}")

Customers DataFrame - Nulls in customer_id: 0
Orders DataFrame - Nulls in order_id: 0
Orders DataFrame - Nulls in customer_id: 0
Products DataFrame - Nulls in product_id: 0


In [54]:

# Check for null values in all columns of customers_df

print("Null counts for customers_df:")

customers_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in customers_df.columns)).show()

# loop over all columns and check for null values

# Check for null values in all columns of orders_df
print("Null counts for orders_df:")
orders_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in orders_df.columns)).show()

# Check for null values in all columns of products_df
print("Null counts for products_df:")
products_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in products_df.columns)).show()

Null counts for customers_df:
+-----------+----+---+-----+-----------+-------+
|customer_id|name|age|email|preferences|address|
+-----------+----+---+-----+-----------+-------+
|          0|   0|  1|    5|          5|      0|
+-----------+----+---+-----+-----------+-------+

Null counts for orders_df:
+--------+-----------+----------+-----+----------------+------------+
|order_id|customer_id|order_date|items|shipping_address|total_amount|
+--------+-----------+----------+-----+----------------+------------+
|       0|          0|         0|    0|              23|          14|
+--------+-----------+----------+-----+----------------+------------+

Null counts for products_df:
+----------+----+--------+-----+----+
|product_id|name|category|price|tags|
+----------+----+--------+-----+----+
|         0|   0|       0|    0|   0|
+----------+----+--------+-----+----+



For numerical columns, fill the missing values with mean value

In [55]:
from pyspark.sql.functions import mean

# Calculate the mean of the 'age' column in customers_df
mean_age = customers_df.select(mean(col("age"))).collect()[0][0]

# Fill null values in the 'age' column with the calculated mean
customers_df = customers_df.na.fill(mean_age, subset=["age"])

# Calculate the mean of the 'total_amount' column in orders_df
mean_total_amount = orders_df.select(mean(col("total_amount"))).collect()[0][0]

# Fill null values in the 'total_amount' column with the calculated mean
orders_df = orders_df.na.fill(mean_total_amount, subset=["total_amount"])

print("Null counts for customers_df after filling 'age' nulls:")
customers_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in customers_df.columns)).show()

print("Null counts for orders_df after filling 'total_amount' nulls:")
orders_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in orders_df.columns)).show()

Null counts for customers_df after filling 'age' nulls:
+-----------+----+---+-----+-----------+-------+
|customer_id|name|age|email|preferences|address|
+-----------+----+---+-----+-----------+-------+
|          0|   0|  0|    5|          5|      0|
+-----------+----+---+-----+-----------+-------+

Null counts for orders_df after filling 'total_amount' nulls:
+--------+-----------+----------+-----+----------------+------------+
|order_id|customer_id|order_date|items|shipping_address|total_amount|
+--------+-----------+----------+-----+----------------+------------+
|       0|          0|         0|    0|              23|           0|
+--------+-----------+----------+-----+----------------+------------+



- For categorical or string columns, fill the missing values with most frequent value.

In [56]:
from pyspark.sql.functions import mode

# Identify string columns in customers_df
string_cols_customers = [c for c, t in customers_df.dtypes if t == 'string']

# Fill null values in string columns of customers_df with the mode
for col_name in string_cols_customers:
    # Calculate the mode for the current column
    mode_value = customers_df.groupBy(col_name).count().orderBy("count", ascending=False).first()[0]
    # Fill nulls with the mode value
    customers_df = customers_df.na.fill(mode_value, subset=[col_name])

# Identify string columns in orders_df
string_cols_orders = [c for c, t in orders_df.dtypes if t == 'string']

# Fill null values in string columns of orders_df with the mode
for col_name in string_cols_orders:
    # Calculate the mode for the current column
    mode_value = orders_df.groupBy(col_name).count().orderBy("count", ascending=False).first()[0]
    # Fill nulls with the mode value
    orders_df = orders_df.na.fill(mode_value, subset=[col_name])

# Identify string columns in products_df
string_cols_products = [c for c, t in products_df.dtypes if t == 'string']

# Fill null values in string columns of products_df with the mode
for col_name in string_cols_products:
    # Calculate the mode for the current column
    mode_value = products_df.groupBy(col_name).count().orderBy("count", ascending=False).first()[0]
    # Fill nulls with the mode value
    products_df = products_df.na.fill(mode_value, subset=[col_name])


print("Null counts for customers_df after filling string column nulls:")
customers_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in customers_df.columns)).show()

print("Null counts for orders_df after filling string column nulls:")
orders_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in orders_df.columns)).show()

print("Null counts for products_df after filling string column nulls:")
products_df.agg(*(sum(col(c).isNull().cast("integer")).alias(c) for c in products_df.columns)).show()

PySparkTypeError: [NOT_BOOL_OR_DICT_OR_FLOAT_OR_INT_OR_STR] Argument `value` should be a bool, dict, float, int or str, got NoneType.